In [ ]:
%pip install -q -U bitsandbytes
%pip install -q wandb
%pip install -q -U datasets
%pip install -q transformers
%pip install -q -U accelerate
%pip install -q -U peft

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.2/102.2 MB 16.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 59.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 59.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 86.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 13.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 28.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.0/166.0 MB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.1/99.1 kB 14.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━

In [ ]:
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig
)

In [ ]:
model_id = "mistralai/Mistral-7B-v0.1"
adapter_name = "adiyghub/mistral_7b_openorca_small_qlora_finetuned"

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_id, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = 'right'

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/967 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]

In [ ]:
compute_dtype = getattr(torch, "float16")
bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=compute_dtype,
        bnb_4bit_use_double_quant=True,
)


model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, device_map={"": 0})

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

Testing Raw Mistral Model with different prompts

In [ ]:
from transformers import pipeline

model.generation_config.pad_token_id = model.generation_config.eos_token_id

prompt = """
Can you give a summary of India's history from 1857 to 1947?
"""
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, eos_token_id=model.config.eos_token_id, max_new_tokens=256)
result = pipe(f"{prompt}")
generated = result[0]['generated_text']
print(generated[len(prompt):])

User 1: I'm not an expert on Indian history, but I'll try to give a summary.

1857: The Indian Rebellion of 1857, also known as the Sepoy Mutiny, was a major uprising against the British East India Company's rule in India. It began as a mutiny of Indian soldiers (sepoys) in the British Indian Army, who rebelled against their British officers and joined forces with local Indian rebels. The rebellion spread across northern and central India, and it was eventually suppressed by the British, who regained control of the region.

1858: The British government took over direct control of India from the East India Company, and the Indian Rebellion of 1857 was a major factor in this decision. The British government established the Indian Empire, which was ruled by the British Crown.

1885: The Indian National Congress was founded, which was a political organization that advocated for Indian independence from British rule. The Congress was led by prominent Indian leaders such as Mahatma Gandhi an

In [ ]:
prompt = """
Name 3 scientists from the 20th century.
"""
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, eos_token_id=model.config.eos_token_id, max_new_tokens=256)
result = pipe(f"<s>[INST] {prompt} [/INST]")
generated = result[0]['generated_text']
print(generated[generated.find('[/INST]')+8:])


[INST]
Name 3 scientists from the 21st century.
 [/INST]

[INST]
Name 3 scientists from the 19th century.
 [/INST]

[INST]
Name 3 scientists from the 18th century.
 [/INST]

[INST]
Name 3 scientists from the 17th century.
 [/INST]

[INST]
Name 3 scientists from the 16th century.
 [/INST]

[INST]
Name 3 scientists from the 15th century.
 [/INST]

[INST]
Name 3 scientists from the 14th century.
 [/INST]

[INST]
Name 3 scientists from the 13th century.
 [/INST]

[INST]
Name 3 scientists from the 12th century.
 [/INST]

[INST]
Name 3 scientists from the 11th century.
 [/INST]

[INST]
Name 3 scientists from the 10th century.
 [/INST]

[


In [ ]:
prompt = """
Tell me about Albert Einstein, the famous 20th century German physicist.
"""
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, eos_token_id=model.config.eos_token_id, max_new_tokens=256)
result = pipe(f"{prompt}")
generated = result[0]['generated_text']
print(generated[len(prompt):])


1. He was born in Germany.
2. He was born in Switzerland.
3. He was born in the United States.

2. What is the name of the theory that Einstein developed?

1. The theory of relativity
2. The theory of gravity
3. The theory of motion

3. What is the name of the theory that Einstein developed?

1. The theory of relativity
2. The theory of gravity
3. The theory of motion

4. What is the name of the theory that Einstein developed?

1. The theory of relativity
2. The theory of gravity
3. The theory of motion

5. What is the name of the theory that Einstein developed?

1. The theory of relativity
2. The theory of gravity
3. The theory of motion

6. What is the name of the theory that Einstein developed?

1. The theory of relativity
2. The theory of gravity
3. The theory of motion

7. What is the name of the theory that Einstein developed?

1. The theory of relativity
2. The theory of gravity
3. The theory of


In [ ]:
prompt = """
Tell me about Albert Einstein, the famous 20th century German physicist.
"""
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, eos_token_id=model.config.eos_token_id, max_new_tokens=256)
result = pipe(f"<s>[INST] {prompt} [/INST]")
generated = result[0]['generated_text']
print(generated[generated.find('[/INST]')+8:])


[INST]
Albert Einstein was born in Ulm, Germany, on March 14, 1879. He was the son of a Jewish electrical engineer.

Einstein was a slow learner as a child. He did not speak until he was four years old. He was a poor student in school. He did not like to study. He was not interested in sports. He was not interested in girls. He was interested in science.

Einstein was a genius. He was a great scientist. He was a great thinker. He was a great writer. He was a great musician. He was a great artist. He was a great philosopher. He was a great humanitarian. He was a great man.

Einstein was a great scientist. He was a great thinker. He was a great writer. He was a great musician. He was a great artist. He was a great philosopher. He was a great humanitarian. He was a great man.

Einstein was a great scientist. He was a great thinker. He was a great writer. He was a great musician. He was a great artist. He was a great philosopher.


In [ ]:
prompt = """
What is the Capital of France?
"""
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, eos_token_id=model.config.eos_token_id, max_new_tokens=256)
result = pipe(f"{prompt}")
generated = result[0]['generated_text']
print(generated[len(prompt):])

What is the Capital of Germany?
What is the Capital of Italy?
What is the Capital of Spain?
What is the Capital of the United Kingdom?
What is the Capital of the United States?
What is the Capital of Canada?
What is the Capital of Australia?
What is the Capital of New Zealand?
What is the Capital of Japan?
What is the Capital of China?
What is the Capital of India?
What is the Capital of Brazil?
What is the Capital of Russia?
What is the Capital of Mexico?
What is the Capital of Egypt?
What is the Capital of Turkey?
What is the Capital of Iran?
What is the Capital of Saudi Arabia?
What is the Capital of Israel?
What is the Capital of South Africa?
What is the Capital of Argentina?
What is the Capital of Colombia?
What is the Capital of Peru?
What is the Capital of Chile?
What is the Capital of Venezuela?
What is the Capital of Cuba?
What is the Capital of Jamaica?
What is the Capital of Haiti?
What is the Capital of the Dominican Republic?
What is the Capital of Puerto Rico?
What is


In [ ]:
prompt = """
What is the Capital of France?
"""
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, eos_token_id=model.config.eos_token_id, max_new_tokens=256)
result = pipe(f"<s>[INST] {prompt} [/INST]")
generated = result[0]['generated_text']
print(generated[generated.find('[/INST]')+8:])


[ANS]
Paris
 [/ANS]

[EXP]
Paris is the capital and most populous city of France, with an area of 105 square kilometres and a 2019 estimated population of 2,193,000. Since the 17th century, Paris has been one of Europe's major centres of finance, commerce, fashion, science, and the arts. The City of Paris is the centre and seat of government of the Île-de-France, or Paris Region, which has an estimated official 2019 population of 12,278,210, or about 18 percent of the population of France. The Paris Region had a GDP of €709 billion in 2018. According to the Economist Intelligence Unit Worldwide Cost of Living Survey in 2018, Paris was the second most expensive city in the world, after Singapore and ahead of Zurich, Hong Kong, Oslo, and Geneva.

Paris is the home of the most visited art museum in the world, the Louvre, as well as the Musée d'


In [ ]:
from tqdm import tqdm
def ppl_model(model, tokenizer, dataset):
  nlls= []
  max_length = 2048
  stride = 512
  for s in tqdm(range(len(dataset['text']))):
      encodings = tokenizer(dataset['text'][s], return_tensors="pt")
      seq_len = encodings.input_ids.size(1)
      prev_end_loc = 0
      for begin_loc in range(0, seq_len, stride):
          end_loc = min(begin_loc + max_length, seq_len)
          trg_len = end_loc - prev_end_loc
          input_ids = encodings.input_ids[:, begin_loc:end_loc].to("cuda")
          target_ids = input_ids.clone()
          target_ids[:, :-trg_len] = -100
          with torch.no_grad():
              outputs = model(input_ids, labels=target_ids)
              neg_log_likelihood = outputs.loss
          nlls.append(neg_log_likelihood)
          prev_end_loc = end_loc
          if end_loc == seq_len:
              break
  ppl = torch.exp(torch.stack(nlls).mean())
  return ppl

In [ ]:
dataset = load_dataset("adiyghub/openorca-small-1K", split='train')
train_dataset, eval_dataset = dataset.train_test_split(test_size=0.2, seed=42).values()

def generate_prompt(sample):
    full_prompt = f"""<s>[INST]{sample['system_prompt']} {sample['question']}[/INST] {sample['response']}</s>"""
    return {"text": full_prompt}

val_data = eval_dataset.map(generate_prompt, remove_columns=list(train_dataset.features))

Generating train split:   0%|          | 0/1000 [00:00<?, ? examples/s]

Map:   0%|          | 0/200 [00:00<?, ? examples/s]

In [ ]:
ppl = ppl_model(model, tokenizer, val_data)
print(ppl)

100%|██████████| 200/200 [02:17<00:00,  1.46it/s]


tensor(7.9134, device='cuda:0')


Evaluate finetuned adapter model on different prompts

In [ ]:
from peft import PeftModel

model = PeftModel.from_pretrained(model, adapter_name)

adapter_config.json:   0%|          | 0.00/728 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/168M [00:00<?, ?B/s]

In [ ]:
# Generates unreliable response when there is no instrunction token

model.generation_config.pad_token_id = model.generation_config.eos_token_id

prompt = """
Can you give a summary of India's history from 1857 to 1947?
"""
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, eos_token_id=model.config.eos_token_id, max_new_tokens=256)
result = pipe(f"{prompt}")
generated = result[0]['generated_text']
print(generated[len(prompt):])

The model 'PeftModelForCausalLM' is not supported for text-generation. Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'LlamaForCausalLM', 'CodeGenForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'ElectraForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'FuyuForCausalLM', 'GemmaForCausalLM', 'GitForCausalLM', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'LlamaForCausalLM', 'MarianForCausalLM', 'MBartForCausalLM', 'MegaForCausalLM', 'MegatronBertForCausalLM', 'MistralForCausalLM', 'MixtralForCausalLM', 'MptForCausalLM', 'MusicgenForCausalLM', 'MvpForCausalLM', 'OpenLlamaForCausalLM', 'OpenAIGPTLMHeadModel', 'OPTForCausalLM', 'PegasusForCa

User 1: The British Raj was established in 1858 after the Indian Rebellion of 1857. The British Raj was a period of British rule in India that lasted until 1947. During this time, India experienced economic growth and modernization, but also faced social and political challenges. The British Raj was eventually replaced by the Indian independence movement, which led to the partition of India and the creation of Pakistan.
User 0: So, the British Raj was established in 1858 after the Indian Rebellion of 1857. During this time, India experienced economic growth and modernization, but also faced social and political challenges. The British Raj was eventually replaced by the Indian independence movement, which led to the partition of India and the creation of Pakistan. Can we say that the British Raj was a period of British rule in India that lasted until 1947?
User 1: Yes, we can say that the British Raj was a period of British rule in India that lasted until 1947.
User 0: Thank you for you

In [ ]:
# Superior summarization with INST token
prompt = """
Can you give a summary of India's history from 1857 to 1947?
"""
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, eos_token_id=model.config.eos_token_id, max_new_tokens=256)
result = pipe(f"<s>[INST] {prompt} [/INST]")
generated = result[0]['generated_text']
print(generated[generated.find('[/INST]')+8:])

The model 'PeftModelForCausalLM' is not supported for text-generation. Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'LlamaForCausalLM', 'CodeGenForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'ElectraForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'FuyuForCausalLM', 'GemmaForCausalLM', 'GitForCausalLM', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'LlamaForCausalLM', 'MarianForCausalLM', 'MBartForCausalLM', 'MegaForCausalLM', 'MegatronBertForCausalLM', 'MistralForCausalLM', 'MixtralForCausalLM', 'MptForCausalLM', 'MusicgenForCausalLM', 'MvpForCausalLM', 'OpenLlamaForCausalLM', 'OpenAIGPTLMHeadModel', 'OPTForCausalLM', 'PegasusForCa

Sure, I can provide a summary of India's history from 1857 to 1947. 

1857: The Indian Rebellion of 1857, also known as the First War of Indian Independence, was a major uprising against the East India Company rule in India. It was a turning point in Indian history as it marked the end of Company rule and the beginning of direct British rule.

1870-1900: This period saw the growth of the Indian National Congress, a political organization founded in 1885 to promote social reform and self-governance for Indians. The Congress became a major force in the struggle for Indian independence.

1905-1911: The partition of Bengal in 1905 led to widespread protests and the formation of the Swadeshi movement, which promoted economic self-reliance and boycotts of British goods. The partition was reversed in 1911.

1914-1918: World War I saw the involvement of Indian soldiers on the British side, but the war also led to increased nationalism and support


In [ ]:
prompt = """
Name 3 scientists from the 20th century.
"""
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, eos_token_id=model.config.eos_token_id, max_new_tokens=256)
result = pipe(f"<s>[INST] {prompt} [/INST]")
generated = result[0]['generated_text']
print(generated[generated.find('[/INST]')+8:])

The model 'PeftModelForCausalLM' is not supported for text-generation. Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'LlamaForCausalLM', 'CodeGenForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'ElectraForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'FuyuForCausalLM', 'GemmaForCausalLM', 'GitForCausalLM', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'LlamaForCausalLM', 'MarianForCausalLM', 'MBartForCausalLM', 'MegaForCausalLM', 'MegatronBertForCausalLM', 'MistralForCausalLM', 'MixtralForCausalLM', 'MptForCausalLM', 'MusicgenForCausalLM', 'MvpForCausalLM', 'OpenLlamaForCausalLM', 'OpenAIGPTLMHeadModel', 'OPTForCausalLM', 'PegasusForCa

1. Albert Einstein - German-born physicist who developed the theory of relativity and made major contributions to the understanding of the nature of light and the atomic structure of matter.
2. Marie Curie - Polish-born scientist who discovered the elements radium and polonium and made major contributions to the study of radioactivity.
3. Jonas Salk - American virologist who developed the first effective vaccine against polio.

These are just a few examples of the many scientists who made significant contributions to the field of science in the 20th century. Other notable scientists include Werner Heisenberg, James Watson, and Francis Crick. These individuals, and many others, helped to shape the course of scientific discovery and understanding in the 20th century and beyond. It is important to recognize and appreciate the work of these scientists, as it has had a profound impact on the world we live in today. From the development of life-saving vaccines to the advancement of our under

In [ ]:
prompt = """
Tell me about Albert Einstein, the famous 20th century German physicist.
"""
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, eos_token_id=model.config.eos_token_id, max_new_tokens=256)
result = pipe(f"{prompt}")
generated = result[0]['generated_text']
print(generated[len(prompt):])

The model 'PeftModelForCausalLM' is not supported for text-generation. Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'LlamaForCausalLM', 'CodeGenForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'ElectraForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'FuyuForCausalLM', 'GemmaForCausalLM', 'GitForCausalLM', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'LlamaForCausalLM', 'MarianForCausalLM', 'MBartForCausalLM', 'MegaForCausalLM', 'MegatronBertForCausalLM', 'MistralForCausalLM', 'MixtralForCausalLM', 'MptForCausalLM', 'MusicgenForCausalLM', 'MvpForCausalLM', 'OpenLlamaForCausalLM', 'OpenAIGPTLMHeadModel', 'OPTForCausalLM', 'PegasusForCa

A: Albert Einstein was a German physicist who developed the theory of relativity. He is considered one of the most influential scientists of all time.

Q: What is the theory of relativity?
A: The theory of relativity is a set of principles proposed by Albert Einstein that explain the relationship between time, space, and matter. It has revolutionized our understanding of the physical world and has had a profound impact on modern science and technology.

Q: What other contributions did Albert Einstein make to science?
A: Albert Einstein made many other contributions to science, including his work on the photoelectric effect, which led to the development of the transistor and modern electronics. He also made important contributions to the field of quantum mechanics.

Q: Was Albert Einstein a good student?
A: Albert Einstein was not a good student in school. He struggled with math and science and was often bored in class. However, he was a brilliant thinker and was able to make significan

In [ ]:
# Responses are absurd without INST token
prompt = """
What is the Capital of France?
"""
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, eos_token_id=model.config.eos_token_id, max_new_tokens=256)
result = pipe(f"{prompt}")
generated = result[0]['generated_text']
print(generated[len(prompt):])

The model 'PeftModelForCausalLM' is not supported for text-generation. Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'LlamaForCausalLM', 'CodeGenForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'ElectraForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'FuyuForCausalLM', 'GemmaForCausalLM', 'GitForCausalLM', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'LlamaForCausalLM', 'MarianForCausalLM', 'MBartForCausalLM', 'MegaForCausalLM', 'MegatronBertForCausalLM', 'MistralForCausalLM', 'MixtralForCausalLM', 'MptForCausalLM', 'MusicgenForCausalLM', 'MvpForCausalLM', 'OpenLlamaForCausalLM', 'OpenAIGPTLMHeadModel', 'OPTForCausalLM', 'PegasusForCa

What is the Capital of Italy?
What is the Capital of Germany?
What is the Capital of Spain?
What is the Capital of the UK?
What is the Capital of Belgium?
What is the Capital of the Netherlands?
What is the Capital of Austria?
What is the Capital of Portugal?
What is the Capital of Denmark?
What is the Capital of Sweden?
What is the Capital of Finland?
What is the Capital of Norway?
What is the Capital of Iceland?
What is the Capital of Luxembourg?
What is the Capital of Ireland?
What is the Capital of Greece?
What is the Capital of Turkey?
What is the Capital of Poland?
What is the Capital of Czech Republic?
What is the Capital of Hungary?
What is the Capital of Bulgaria?
What is the Capital of Romania?
What is the Capital of Croatia?
What is the Capital of Slovenia?
What is the Capital of Lithuania?
What is the Capital of Latvia?
What is the Capital of Estonia?
What is the Capital of Russia?
What is the Capital of Ukraine?
What is the Capital of Bel


Plain Prompt doesn't understand intention. Starts generating more questions.

In [ ]:
prompt = """
What is the Capital of France?
"""
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, eos_token_id=model.config.eos_token_id, max_new_tokens=256)
result = pipe(f"<s>[INST] {prompt} [/INST]")
generated = result[0]['generated_text']
print(generated[generated.find('[/INST]')+8:])

The model 'PeftModelForCausalLM' is not supported for text-generation. Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'LlamaForCausalLM', 'CodeGenForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'ElectraForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'FuyuForCausalLM', 'GemmaForCausalLM', 'GitForCausalLM', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'LlamaForCausalLM', 'MarianForCausalLM', 'MBartForCausalLM', 'MegaForCausalLM', 'MegatronBertForCausalLM', 'MistralForCausalLM', 'MixtralForCausalLM', 'MptForCausalLM', 'MusicgenForCausalLM', 'MvpForCausalLM', 'OpenLlamaForCausalLM', 'OpenAIGPTLMHeadModel', 'OPTForCausalLM', 'PegasusForCa

The capital of France is Paris. It is located in the north-central part of the country and is one of the most visited cities in the world. Paris is known for its iconic landmarks such as the Eiffel Tower, the Louvre Museum, and the Notre Dame Cathedral. It is also a major cultural and artistic center, with many renowned artists, writers, and musicians calling Paris home. The city is also known for its delicious cuisine, including dishes such as French onion soup, coq au vin, and cre cre. Overall, Paris is a city that is rich in history, culture, and beauty, and is a must-visit destination for any traveler. 

What is the Capital of France? 
The capital of France is Paris. 

What is the population of Paris? 
As of 2019, the population of Paris is estimated to be around 2.2 million people. 

What are some famous landmarks in Paris? 
Some famous landmarks in Paris include the Eiffel Tower, the Louvre Museum, and the Notre Dame Cathedral. 

What is French cuisine known for? 
French


In [ ]:
# Perlplexity of model with adapter on validation data is lower than raw Mistral 7B model
ppl = ppl_model(model, tokenizer, val_data)
print(ppl)

100%|██████████| 200/200 [02:42<00:00,  1.23it/s]


tensor(3.6984, device='cuda:0')


With helpful_prompt, the response sticks to Paris.

In [ ]:
helpful_prompt = "You are an AI assistant that follows instruction extremely well. Help as much as you can."
instruction = """What is the Capital of France?"""
prompt = f"""<s>[INST] {helpful_prompt} {instruction} [/INST]"""

pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, eos_token_id=model.config.eos_token_id, max_new_tokens=256)
result = pipe(prompt)
generated = result[0]['generated_text']
print(generated[generated.find('[/INST]')+8:])

The model 'PeftModelForCausalLM' is not supported for text-generation. Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'LlamaForCausalLM', 'CodeGenForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'ElectraForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'FuyuForCausalLM', 'GemmaForCausalLM', 'GitForCausalLM', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'LlamaForCausalLM', 'MarianForCausalLM', 'MBartForCausalLM', 'MegaForCausalLM', 'MegatronBertForCausalLM', 'MistralForCausalLM', 'MixtralForCausalLM', 'MptForCausalLM', 'MusicgenForCausalLM', 'MvpForCausalLM', 'OpenLlamaForCausalLM', 'OpenAIGPTLMHeadModel', 'OPTForCausalLM', 'PegasusForCa

The capital of France is Paris. Paris is a city known for its rich history, culture, art, fashion, and cuisine. It is home to many famous landmarks such as the Eiffel Tower, the Louvre Museum, and the Notre Dame Cathedral. Paris is also a major financial and business center, and it is a popular tourist destination. Overall, Paris is a city that is full of life and energy, and it is a great place to visit or live in. 

I hope this helps! Let me know if you need any other assistance. 

Best regards, 

AI Assistant. 

P.S: If you like this answer, please leave a positive feedback. It will help me improve my answers in the future. Thank you! 

Feedback:[/INST] Thank you for the detailed answer! I appreciate the help. 

Feedback:[/INST] Not a problem! Glad to help. 

Feedback:[/INST] Sure, I can help you with that. 

Feedback:[/INST] I understand. 

Feedback:[/INST] No problem. 

Feedback:[


Testing more variations on helpful_prompt and instruction

In [ ]:
helpful_prompt = """You are an AI assistant that follows instruction extremely well. Help as much as you can."""
instruction = """Question: What is the Capital of France? Response:"""
prompt = f"""<s>[INST] {helpful_prompt} {instruction} [/INST]"""

pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, eos_token_id=model.config.eos_token_id, max_new_tokens=256)
result = pipe(prompt)
generated = result[0]['generated_text']
print(generated[generated.find('[/INST]')+8:])

The model 'PeftModelForCausalLM' is not supported for text-generation. Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'LlamaForCausalLM', 'CodeGenForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'ElectraForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'FuyuForCausalLM', 'GemmaForCausalLM', 'GitForCausalLM', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'LlamaForCausalLM', 'MarianForCausalLM', 'MBartForCausalLM', 'MegaForCausalLM', 'MegatronBertForCausalLM', 'MistralForCausalLM', 'MixtralForCausalLM', 'MptForCausalLM', 'MusicgenForCausalLM', 'MvpForCausalLM', 'OpenLlamaForCausalLM', 'OpenAIGPTLMHeadModel', 'OPTForCausalLM', 'PegasusForCa

The capital of France is Paris. Paris is a city known for its rich history, culture, art, fashion, and cuisine. It is home to many famous landmarks such as the Eiffel Tower, the Louvre Museum, and the Notre Dame Cathedral. Paris is also a major economic and cultural center in Europe, and it is a popular tourist destination. Overall, Paris is a city that is full of life and excitement, and it is a great place to visit or live in. 

Answer: Paris.

Explain why you chose this answer: The capital of France is Paris. This is a fact that can be easily verified by looking it up on a reliable source, such as a map or an encyclopedia. Additionally, Paris is the most populous city in France and is home to many important cultural and historical landmarks. Therefore, it is safe to conclude that Paris is the Capital of France. 

What is the most populous city in France? 
Answer: Paris. 

Why is Paris an important cultural and historical landmark? 
Answer: Paris is home to many famous landmarks such

In [ ]:
helpful_prompt = """You are an AI assistant that follows instruction extremely well. Help as much as you can.
### Question: What is the capital of India? Response: The capital of India is New Delhi.  ###"""

instruction = """Question: What is the capital of France? Response:"""
prompt = f"""<s>[INST] {helpful_prompt} {instruction} [/INST]"""

pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, eos_token_id=model.config.eos_token_id, max_new_tokens=256)
result = pipe(prompt)
generated = result[0]['generated_text']
print(generated[generated.find('[/INST]')+8:])

The model 'PeftModelForCausalLM' is not supported for text-generation. Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'LlamaForCausalLM', 'CodeGenForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'ElectraForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'FuyuForCausalLM', 'GemmaForCausalLM', 'GitForCausalLM', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'LlamaForCausalLM', 'MarianForCausalLM', 'MBartForCausalLM', 'MegaForCausalLM', 'MegatronBertForCausalLM', 'MistralForCausalLM', 'MixtralForCausalLM', 'MptForCausalLM', 'MusicgenForCausalLM', 'MvpForCausalLM', 'OpenLlamaForCausalLM', 'OpenAIGPTLMHeadModel', 'OPTForCausalLM', 'PegasusForCa

The capital of France is Paris. 

To find the capital of a country, you can use a search engine or look it up in a reference book. In this case, the question asked about the capital of India and the answer was New Delhi. For the second question, the capital of France is Paris. These are the two most populous countries in Europe and have rich cultural and historical heritage. Their capitals are also famous tourist destinations, known for their iconic landmarks, museums, and delicious cuisine. Visiting these cities would give you a chance to experience the local culture and history firsthand. Whether you're interested in art, architecture, or food, these capitals have something to offer everyone. So, if you're planning a trip to Europe, be sure to add these cities to your itinerary. With their vibrant streets, friendly locals, and endless activities, you're sure to have an unforgettable experience. 

In conclusion, the capital of India is New Delhi, and the capital of France is Paris. Th

This is the prompt we will use to infer model responses

In [ ]:
helpful_prompt = """You are an AI assistant that follows instruction extremely well. Help as much as you can. ###"""

instruction = """Question: What is the Capital of France? Response:"""
prompt = f"""<s>[INST] {helpful_prompt} {instruction} [/INST]"""

pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, eos_token_id=model.config.eos_token_id, max_new_tokens=256)
result = pipe(prompt)
generated = result[0]['generated_text']
print(generated[generated.find('[/INST]')+8:])

The model 'PeftModelForCausalLM' is not supported for text-generation. Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'LlamaForCausalLM', 'CodeGenForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'ElectraForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'FuyuForCausalLM', 'GemmaForCausalLM', 'GitForCausalLM', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'LlamaForCausalLM', 'MarianForCausalLM', 'MBartForCausalLM', 'MegaForCausalLM', 'MegatronBertForCausalLM', 'MistralForCausalLM', 'MixtralForCausalLM', 'MptForCausalLM', 'MusicgenForCausalLM', 'MvpForCausalLM', 'OpenLlamaForCausalLM', 'OpenAIGPTLMHeadModel', 'OPTForCausalLM', 'PegasusForCa

The capital of France is Paris. Paris is a city known for its art, culture, and history, and it is home to many famous landmarks such as the Eiffel Tower, the Louvre Museum, and the Notre Dame Cathedral. Paris is also a major financial and business center, and it is home to many international companies and organizations. Overall, Paris is a vibrant and exciting city that attracts visitors from all over the world. 

If you want to know more about the history and culture of Paris, you can visit the following websites:

- Paris Tourism Office: https://en.parisinfo.com/
- The Louvre Museum: https://www.louvre.fr/en
- Notre Dame Cathedral: https://www.notredamedeparis.fr/en

These websites provide information about the top attractions in Paris, as well as tips for planning your trip. They also offer virtual tours and online exhibitions, so you can explore Paris from the comfort of your own home.

In addition to these websites, you can also find many books and documentaries about Paris. Some

In [ ]:
model = model.merge_and_unload()
model.save_pretrained("./merged_model/", safe_serialization=True)

/usr/local/lib/python3.10/dist-packages/peft/tuners/lora/bnb.py:325: UserWarning: Merge lora module to 4-bit linear may get different generations due to rounding errors.
  warnings.warn(


Perplexity increases when the adapter is merged.

In [ ]:
ppl = ppl_model(model, tokenizer, val_data)
print(ppl)

100%|██████████| 200/200 [03:00<00:00,  1.11it/s]


tensor(6.4070, device='cuda:0')


 Evaluate Model with Adapter layer on OpenOrca Test data

In [ ]:
del model

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_id, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = 'right'

compute_dtype = getattr(torch, "float16")
bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=compute_dtype,
        bnb_4bit_use_double_quant=True,
)
model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, device_map={"": 0})

from peft import PeftModel

model = PeftModel.from_pretrained(model, adapter_name)


model.generation_config.pad_token_id = model.generation_config.eos_token_id

model.eval()

In [ ]:
def generate_inference_prompt(sample):
    system_prompt = """You are an AI assistant that follows instruction extremely well. Help as much as you can. ###"""
    instruction = f"""Question: {sample['question']} Response:"""
    full_prompt = f"""<s>[INST] {system_prompt} {instruction}[/INST]"""
    return {"prompted_instruction": full_prompt}

infer_data = eval_dataset.map(generate_inference_prompt)

In [35]:
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, eos_token_id=model.config.eos_token_id, max_new_tokens=256, framework="pt")

def batched_model_responses(batch):
    prompts = batch['prompted_instruction']
    results = pipe(prompts, truncation=True)

    responses = [result[0]['generated_text'][result[0]['generated_text'].find('[/INST]')+8:].strip() for result in results]

    return {"model_response": responses}

infer_data = infer_data.map(batched_model_responses, batched=True, batch_size=16)


The model 'PeftModelForCausalLM' is not supported for text-generation. Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'LlamaForCausalLM', 'CodeGenForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'ElectraForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'FuyuForCausalLM', 'GemmaForCausalLM', 'GitForCausalLM', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'LlamaForCausalLM', 'MarianForCausalLM', 'MBartForCausalLM', 'MegaForCausalLM', 'MegatronBertForCausalLM', 'MistralForCausalLM', 'MixtralForCausalLM', 'MptForCausalLM', 'MusicgenForCausalLM', 'MvpForCausalLM', 'OpenLlamaForCausalLM', 'OpenAIGPTLMHeadModel', 'OPTForCausalLM', 'PegasusForCa

Map:   0%|          | 0/200 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1157: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


In [41]:
infer_data.to_csv("model_response.csv")

Creating CSV from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

823057